In [1]:
# prompt: check if I have any gpus available

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:
# print gpu details
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  for gpu in gpus:
    details = tf.config.experimental.get_device_details(gpu)
    print(details)

else:
  print("No GPUs available")


{'compute_capability': (8, 6), 'device_name': 'NVIDIA RTX A2000 12GB'}


In [14]:
!pip install pydrive oauth2client

In [ ]:
!pip install gdown
import gdown

In [ ]:
file_id = "1har8IjJSuCVZFd4U01WD_pdwP9iEnCrL"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "/content/output_file.csv", quiet=False)


In [96]:
import pandas as pd
data = pd.read_csv('/content/output_file.csv')
print(data.head())

         DATE  PRECIPITATION  MAX_TEMP  MIN_TEMP  AVG_WIND_SPEED  \
0  1984-01-01            0.0      79.0      51.0            4.70   
1  1984-01-02            0.0      71.0      46.0            5.59   
2  1984-01-03            0.0      70.0      47.0            5.37   
3  1984-01-04            0.0      76.0      45.0            4.70   
4  1984-01-05            0.0      74.0      49.0            5.14   

   FIRE_START_DAY  YEAR  TEMP_RANGE  WIND_TEMP_RATIO  MONTH  SEASON  \
0           False  1984        28.0         0.059494      1  Winter   
1           False  1984        25.0         0.078732      1  Winter   
2           False  1984        23.0         0.076714      1  Winter   
3           False  1984        31.0         0.061842      1  Winter   
4           False  1984        25.0         0.069459      1  Winter   

   LAGGED_PRECIPITATION  LAGGED_AVG_WIND_SPEED  DAY_OF_YEAR  
0                   0.0                  4.700            1  
1                   0.0                 

In [97]:
import pandas as pd
import scipy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [98]:
data.describe()

,PRECIPITATION,MAX_TEMP,MIN_TEMP,AVG_WIND_SPEED,YEAR,TEMP_RANGE,WIND_TEMP_RATIO,MONTH,LAGGED_PRECIPITATION,LAGGED_AVG_WIND_SPEED,DAY_OF_YEAR
count,14987.000000,14987.000000,14987.000000,14976.000000,14988.000000,14987.000000,14976.000000,14988.000000,14988.000000,14988.000000,14988.000000
mean,0.032315,70.534997,56.494095,7.435098,2004.016813,14.040902,0.107019,6.518281,0.226188,7.434198,182.992994
std,0.179544,7.263447,6.767685,2.129985,11.843342,5.995327,0.035630,3.451037,0.648705,1.387849,105.523627
min,0.000000,50.000000,33.000000,1.790000,1984.000000,2.000000,0.023553,1.000000,0.000000,3.227143,1.000000
25%,0.000000,65.000000,51.000000,6.040000,1994.000000,10.000000,0.085238,4.000000,0.000000,6.518571,92.000000
50%,0.000000,70.000000,57.000000,7.160000,2004.000000,12.000000,0.102222,7.000000,0.000000,7.478571,183.000000
75%,0.000000,75.000000,62.000000,8.500000,2014.000000,17.000000,0.120462,10.000000,0.060000,8.278571,274.000000
max,4.530000,106.000000,77.000000,26.170000,2025.000000,41.000000,0.459123,12.000000,8.180000,13.932857,366.000000


In [109]:
print(data.isnull().sum())

DATE                      0
PRECIPITATION             1
MAX_TEMP                  1
MIN_TEMP                  1
AVG_WIND_SPEED           12
FIRE_START_DAY            0
YEAR                      0
TEMP_RANGE                1
WIND_TEMP_RATIO          12
MONTH                     0
SEASON                    0
LAGGED_PRECIPITATION      0
LAGGED_AVG_WIND_SPEED     0
DAY_OF_YEAR               0
dtype: int64


In [100]:
print(data)

             DATE  PRECIPITATION  MAX_TEMP  MIN_TEMP  AVG_WIND_SPEED  \
0      1984-01-01            0.0      79.0      51.0            4.70   
1      1984-01-02            0.0      71.0      46.0            5.59   
2      1984-01-03            0.0      70.0      47.0            5.37   
3      1984-01-04            0.0      76.0      45.0            4.70   
4      1984-01-05            0.0      74.0      49.0            5.14   
...           ...            ...       ...       ...             ...   
14983  2025-01-08            0.0      73.0      53.0           10.51   
14984  2025-01-09            0.0      68.0      46.0            4.92   
14985  2025-01-10            0.0      70.0      46.0            3.58   
14986  2025-01-11            0.0      66.0      46.0             NaN   
14987  2025-01-12            0.0      69.0      46.0             NaN   

       FIRE_START_DAY  YEAR  TEMP_RANGE  WIND_TEMP_RATIO  MONTH  SEASON  \
0               False  1984        28.0         0.059494    

In [144]:
#Feature engineering
y = data["FIRE_START_DAY"]
X = data.drop("FIRE_START_DAY", axis=1)
X = X.drop("DATE", axis=1)

In [145]:
print(X)

       PRECIPITATION  MAX_TEMP  MIN_TEMP  AVG_WIND_SPEED  YEAR  TEMP_RANGE  \
0                0.0      79.0      51.0            4.70  1984        28.0   
1                0.0      71.0      46.0            5.59  1984        25.0   
2                0.0      70.0      47.0            5.37  1984        23.0   
3                0.0      76.0      45.0            4.70  1984        31.0   
4                0.0      74.0      49.0            5.14  1984        25.0   
...              ...       ...       ...             ...   ...         ...   
14983            0.0      73.0      53.0           10.51  2025        20.0   
14984            0.0      68.0      46.0            4.92  2025        22.0   
14985            0.0      70.0      46.0            3.58  2025        24.0   
14986            0.0      66.0      46.0             NaN  2025        20.0   
14987            0.0      69.0      46.0             NaN  2025        23.0   

       WIND_TEMP_RATIO  MONTH  SEASON  LAGGED_PRECIPITATION  \


In [146]:
#Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [147]:
from sklearn.impute import SimpleImputer
x_train_values = ["AVG_WIND_SPEED", "PRECIPITATION", "MAX_TEMP", "MIN_TEMP", "TEMP_RANGE", "WIND_TEMP_RATIO"]
def cleanX(val):
  frontage_train = X_train[[val]]

  # (2) Instantiate a SimpleImputer with strategy="median"
  imputer = SimpleImputer(strategy='median')

  # (3) Fit the imputer on frontage_train
  imputer.fit(frontage_train)

  # (4) Transform frontage_train using the imputer and
  # assign the result to frontage_imputed_train
  frontage_imputed_train = imputer.transform(frontage_train)

  # (5) Replace value of LotFrontage
  X_train[val] = frontage_imputed_train

for val in x_train_values:
  cleanX(val)


In [148]:
print(X_train.isnull().sum())

PRECIPITATION            0
MAX_TEMP                 0
MIN_TEMP                 0
AVG_WIND_SPEED           0
YEAR                     0
TEMP_RANGE               0
WIND_TEMP_RATIO          0
MONTH                    0
SEASON                   0
LAGGED_PRECIPITATION     0
LAGGED_AVG_WIND_SPEED    0
DAY_OF_YEAR              0
dtype: int64


In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14988 entries, 0 to 14987
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DATE                   14988 non-null  object 
 1   PRECIPITATION          14987 non-null  float64
 2   MAX_TEMP               14987 non-null  float64
 3   MIN_TEMP               14987 non-null  float64
 4   AVG_WIND_SPEED         14976 non-null  float64
 5   FIRE_START_DAY         14988 non-null  bool   
 6   YEAR                   14988 non-null  int64  
 7   TEMP_RANGE             14987 non-null  float64
 8   WIND_TEMP_RATIO        14976 non-null  float64
 9   MONTH                  14988 non-null  int64  
 10  SEASON                 14988 non-null  object 
 11  LAGGED_PRECIPITATION   14988 non-null  float64
 12  LAGGED_AVG_WIND_SPEED  14988 non-null  float64
 13  DAY_OF_YEAR            14988 non-null  int64  
dtypes: bool(1), float64(8), int64(3), object(2)
memory usa

In [150]:
# (0) import OneHotEncoder from sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder

# (1) Create a variable fireplace_qu_train
# extracted from X_train
# (double brackets due to shape expected by OHE)
season_train = X_train[["SEASON"]]

# (2) Instantiate a OneHotEncoder with categories="auto",
# sparse=False, and handle_unknown="ignore"
ohe = OneHotEncoder(categories='auto', sparse_output = False, handle_unknown = 'ignore')

# (3) Fit the encoder on season_train
ohe.fit(season_train)

# (4) Transform season_train using the encoder and
# assign the result to season_encoded_train
season_encoded_train = ohe.transform(season_train)

# (5a) Make the transformed data into a dataframe
season_encoded_train = pd.DataFrame(
    # Pass in NumPy array
    season_encoded_train,
    # Set the column names to the categories found by OHE
    columns=ohe.categories_[0],
    # Set the index to match X_train's index
    index=X_train.index
)

# (5b) Drop original Season column
X_train.drop("SEASON", axis=1, inplace=True)

# (5c) Concatenate the new dataframe with current X_train
X_train = pd.concat([X_train, season_encoded_train], axis=1)

In [151]:
print(X_train)

       PRECIPITATION  MAX_TEMP  MIN_TEMP  AVG_WIND_SPEED  YEAR  TEMP_RANGE  \
7862             0.0      73.0      64.0            6.93  2005         9.0   
2952             0.0      79.0      52.0            6.93  1992        27.0   
11520            0.0      73.0      64.0            8.50  2015         9.0   
13117            0.0      61.0      44.0            6.04  2019        17.0   
13104            0.0      93.0      55.0            3.58  2019        38.0   
...              ...       ...       ...             ...   ...         ...   
5191             0.0      65.0      56.0            8.50  1998         9.0   
13418            0.0      77.0      64.0            4.70  2020        13.0   
5390             0.0      72.0      57.0            7.61  1998        15.0   
860              0.0      77.0      55.0            8.50  1986        22.0   
7270             0.0      76.0      48.0            5.59  2003        28.0   

       WIND_TEMP_RATIO  MONTH  LAGGED_PRECIPITATION  LAGGED_AVG

In [152]:
#Train-test split
from sklearn.linear_model import LinearRegression

#Linear Regression with scikitlearn
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [154]:
#Preprocessing the training data
from sklearn.impute import SimpleImputer
x_test_values = ["AVG_WIND_SPEED", "PRECIPITATION", "MAX_TEMP", "MIN_TEMP", "TEMP_RANGE", "WIND_TEMP_RATIO"]
def cleanX(val):
  frontage_test = X_test[[val]]

  # (2) Instantiate a SimpleImputer with strategy="median"
  imputer = SimpleImputer(strategy='median')

  # (3) Fit the imputer on frontage_train
  imputer.fit(frontage_test)

  # (4) Transform frontage_train using the imputer and
  # assign the result to frontage_imputed_train
  frontage_imputed_test= imputer.transform(frontage_test)

  # (5) Replace value of LotFrontage
  X_test[val] = frontage_imputed_test

for val in x_test_values:
  cleanX(val)


In [155]:
# (0) import OneHotEncoder from sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder

# (1) Create a variable fireplace_qu_train
# extracted from X_test
# (double brackets due to shape expected by OHE)
season_test = X_test[["SEASON"]]

# (2) Instantiate a OneHotEncoder with categories="auto",
# sparse=False, and handle_unknown="ignore"
ohe = OneHotEncoder(categories='auto', sparse_output = False, handle_unknown = 'ignore')

# (3) Fit the encoder on season_test
ohe.fit(season_test)

# (4) Transform season_test using the encoder and
# assign the result to season_encoded_test
season_encoded_test = ohe.transform(season_test)

# (5a) Make the transformed data into a dataframe
season_encoded_test = pd.DataFrame(
    # Pass in NumPy array
    season_encoded_test,
    # Set the column names to the categories found by OHE
    columns=ohe.categories_[0],
    # Set the index to match X_train's index
    index=X_test.index
)

# (5b) Drop original Season column
X_test.drop("SEASON", axis=1, inplace=True)

# (5c) Concatenate the new dataframe with current X_test
X_test = pd.concat([X_test, season_encoded_test], axis=1)

In [156]:
#Make predictions

model.score(X_train, y_train)
model.score(X_test, y_test)

0.2788260403930224

In [ ]:
print(X_test)

In [170]:
#Random Forest with scikitlearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score


rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# 4. Train the model
rf_classifier.fit(X_train, y_train)

# 5. Make predictions on the test set
# predictions = rf_classifier.predict(X_test)
# print(f"Prediction: {predictions}")

feature_names = [
    "PRECIPITATION", "MAX_TEMP", "MIN_TEMP", "AVG_WIND_SPEED", "YEAR",
    "TEMP_RANGE", "WIND_TEMP_RATIO", "MONTH", "LAGGED_PRECIPITATION",
    "LAGGED_AVG_WIND_SPEED", "DAY_OF_YEAR", "Fall", "Spring", "Summer", "Winter"
]

# Convert X_test to a DataFrame
X_test_df = pd.DataFrame([[0.0, 69.0, 58.0, 6.49, 1989, 11.0, 0.094058, 4, 0.00, 7.285714, 250, 1.0, 0.0, 0.0, 0.0]],
                         columns=feature_names)

# Make predictions
inference = rf_classifier.predict(X_test_df)
print(inference)

probabilities = rf_classifier.predict_proba(X_test_df)

# Print the probability of the prediction being True (class 1)
print(f"Probability of True: {probabilities[0][1]}")


[False]
Probability of True: 0.38


       PRECIPITATION  MAX_TEMP  MIN_TEMP  AVG_WIND_SPEED  YEAR  TEMP_RANGE  \
5992             0.0      76.0      63.0            6.71  2000        13.0   
9981             0.0      77.0      55.0            6.26  2011        22.0   
12717            0.0      73.0      61.0            5.59  2018        12.0   
6518             0.0      68.0      57.0            5.37  2001        11.0   
8883             0.0      95.0      70.0            7.61  2008        25.0   
...              ...       ...       ...             ...   ...         ...   
2656             0.0      73.0      59.0           11.41  1991        14.0   
6753             0.0      71.0      61.0            7.61  2002        10.0   
7484             0.0      74.0      64.0            9.17  2004        10.0   
10950            0.0      78.0      53.0            4.70  2013        25.0   
1934             0.0      69.0      58.0            6.49  1989        11.0   

       WIND_TEMP_RATIO  MONTH  LAGGED_PRECIPITATION  LAGGED_AVG

In [171]:
import joblib

# Save the trained model
joblib.dump(rf_classifier, "rf_model.pkl")

['rf_model.pkl']

In [172]:
from google.colab import files
files.download("rf_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>